In [1]:
import re
import pandas as pd
import numpy as np

In [11]:
normal_ac=pd.read_csv('D:/study/Fake/genuine_accounts/users.csv')
normal_tweet=pd.read_csv('D:/study/Fake/normlfinal.csv')
spam_user1=pd.read_csv('D:/study/Fake/social_spambots_1/users.csv')
spam_user2 = pd.read_csv('D:/study/Fake/social_spambots_2/users.csv')
spam_user3= pd.read_csv('D:/study/Fake/social_spambots_3/users.csv')
spam_user=pd.concat([spam_user1,spam_user2,spam_user3],axis=0)

D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [12]:
normal_tweet['user_id']=normal_tweet['user_id'].astype(int)
print(normal_ac.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3474 entries, 0 to 3473
Data columns (total 42 columns):
id                                    3474 non-null int64
name                                  3473 non-null object
screen_name                           3474 non-null object
statuses_count                        3474 non-null int64
followers_count                       3474 non-null int64
friends_count                         3474 non-null int64
favourites_count                      3474 non-null int64
listed_count                          3474 non-null int64
url                                   1266 non-null object
lang                                  3474 non-null object
time_zone                             2475 non-null object
location                              2365 non-null object
default_profile                       1032 non-null float64
default_profile_image                 13 non-null float64
geo_enabled                           2155 non-null float64
profile_image

In [143]:
from urllib.parse import *
url = "http://hubpages.com\__CLIENT_ERROR__"
result = urlparse(url)
header = '{uri.scheme}'.format(uri=result)
domain = '{uri.netloc}'.format(uri=result)
print(result)

ParseResult(scheme='http', netloc='hubpages.com\\__CLIENT_ERROR__', path='', params='', query='', fragment='')


In [13]:
normal_ac['protected'].value_counts()

1.0    78
Name: protected, dtype: int64

In [14]:
def cleanUser(df):
    df=df[['id','url','followers_count','friends_count'
        ,'created_at','verified','protected']]
    df = df.rename(columns={'id': 'user_id'})
    df['reputation']=round((df['followers_count']/
                              (df['followers_count']
                               +df['friends_count'])), 2)
    df['social']=round((df['friends_count']
                           /df['followers_count']), 2)
    df['created_at']=pd.to_datetime(df['created_at'])
    df['verified'].fillna(0,inplace=True)
    df['protected'].fillna(0,inplace=True)
    df['url'].fillna('no',inplace=True)
    df.set_index(["user_id"], inplace=True)
    
    
    return df

In [15]:
spam_user=cleanUser(spam_user)
normal_ac=cleanUser(normal_ac)
print(normal_ac.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3474 entries, 1502026416 to 2910276853
Data columns (total 8 columns):
url                3474 non-null object
followers_count    3474 non-null int64
friends_count      3474 non-null int64
created_at         3474 non-null datetime64[ns, UTC]
verified           3474 non-null float64
protected          3474 non-null float64
reputation         3474 non-null float64
social             3474 non-null float64
dtypes: datetime64[ns, UTC](1), float64(4), int64(2), object(1)
memory usage: 244.3+ KB
None


In [16]:
# normal_ac.set_index(["user_id"], inplace=True)
print(normal_ac.head())

                               url  followers_count  friends_count  \
user_id                                                              
1502026416                      no              208            332   
2492782375                      no              330            485   
293212315                       no              166            177   
191839658   http://t.co/rGV0HIJGsu             2248            981   
3020965143                      no               21             79   

                          created_at  verified  protected  reputation  social  
user_id                                                                        
1502026416 2013-06-11 11:20:35+00:00       0.0        0.0        0.39    1.60  
2492782375 2014-05-13 10:37:57+00:00       0.0        0.0        0.40    1.47  
293212315  2011-05-04 23:30:37+00:00       0.0        0.0        0.48    1.07  
191839658  2010-09-17 14:02:10+00:00       0.0        0.0        0.70    0.44  
3020965143 2015-02-06 04:10:4

In [17]:
normal=normal_tweet.join(normal_ac,on='user_id')

In [18]:
print(normal.head())

                   id                                               text  \
0  593819412660101121                 Don't. Ok? https://t.co/uTXrJ6zvdN   
1  593814612820119552  RT @SanFranciscoBA: #enoughsenough "Surprise! ...   
2  593659318479687680  RT @sippey: Here's my one paragraph review of ...   
3  593579247110782976  @MMFlint I dont' ask for RTs gen, but my frien...   
4  593578595370438657  Not only$ but acts+messages from people that s...   

   user_id  num_hashtags  num_mentions  num_urls  \
0   678033           0.0           0.0       1.0   
1   678033           1.0           1.0       1.0   
2   678033           0.0           1.0       1.0   
3   678033           0.0           2.0       1.0   
4   678033           1.0           1.0       1.0   

                                             finlurl                     url  \
0            https://www.instagram.com/p/2Gzq09K0MM/  http://t.co/98fWa0t873   
1                                      https://t.co/  http://t.co/98fW

In [19]:
normal['social'].sum()

754592.8100000002

In [20]:
spam_tweets=pd.read_csv('D:/study/Fake/spamfinal.csv')
spam=spam_tweets.join(spam_user,on='user_id')

In [21]:
spam['social'].sum()

inf

In [22]:
normal['isSpam']=0
spam['isSpam']=1
print(spam.head())

                   id                                              text  \
0  532627591686275072  I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws   
1  532627591686275072  I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws   
2  532627591686275072  I Pooh - In silenzio 1968 http://t.co/ahvQxUqTws   
3  532613365307547649  Mother daughter java time http://t.co/iPNFLGY2dA   
4  532613365307547649  Mother daughter java time http://t.co/iPNFLGY2dA   

    user_id  num_hashtags  num_mentions  num_urls  \
0  24858289             0             0         1   
1  24858289             0             0         1   
2  24858289             0             0         1   
3  16195340             0             0         1   
4  16195340             0             0         1   

                                             finlurl                     url  \
0        https://www.youtube.com/watch?v=Y6f6_WiyWjI                      no   
1              http://instagram.com\__CLIENT_ERROR__                  

In [39]:
df=pd.concat([normal,spam],axis=0)

In [40]:
del spam_user,spam_user1,spam_user2,spam_user3,spam_tweets,normal_tweet

NameError: name 'spam_user' is not defined

In [41]:
import string
from nltk.corpus import stopwords
stopwords=stopwords.words('english')
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("english")
def token(text):
    tokens=[]
    remove_spl_char_regex = re.compile('[%s]' % re.escape(string.punctuation))
    text = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*(),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '',
                  text)  # to replace url with ''
    text = remove_spl_char_regex.sub(" ", text)  # Remove special characters
    text = text.lower()
    for word in text.split():
        if word not in stopwords \
                and word not in string.punctuation \
                and len(word) > 2 \
                and word != '``':
            tokens.append(word)
    # stems = [stemmer.stem(t) for t in tokens]
    tweet=' '.join(tokens)
    return tweet

In [42]:
df['text']=df['text'].apply(token)

In [43]:
#remove empty text
mask=df['text'].str.len()==0
df.loc[mask,'text'] =np.nan
df.dropna(axis=0,inplace=True)


In [44]:
# spamList=open("D:\study\python\Spamdetection\spam word.txt").read()
# spamList=spamList.split()
# spamList+='sanfranciscoba'
# print(spamList)
# def spamWordRatio(word):
#     count=0
#     for i in word:
#         if i in spamList:
#             count=count+1
#     return count/len(word)
# df['spamRatio']=df['text'].apply(spamWordRatio)
# print(df['spamRatio'].value_counts())

In [45]:
df = df.rename(columns={'finlurl': 'finalurl'})

In [46]:
def extractDomainFromURL(url):
    """Get domain name from url"""
    from urllib.parse import urlparse
    parsed_uri = urlparse(url)
    header = '{uri.netloc}'.format(uri=parsed_uri)
    domain = '{uri.netloc}'.format(uri=parsed_uri)
    return domain
df['finalurl']=df['finalurl'].apply(extractDomainFromURL)

In [119]:
x=df['finalurl'][0:1]
x="http://"+x
print(x)

1    http://t.co
Name: finalurl, dtype: object


In [47]:
df2=df.copy()

In [37]:
print(len(df2))
print(len(new))

1469835


NameError: name 'new' is not defined

In [48]:

total = pd.DataFrame({'size':df2.groupby('finalurl').size()})
total = total.sort_values(['size'], ascending=False)
total.reset_index(inplace=True)
total['index']=total.index
total.set_index(['finalurl'],inplace=True)
print(total)
df2=df2.join(total,on='finalurl')


                                                      size  index
finalurl                                                         
hubpages.com\__CLIENT_ERROR__                       454996      0
twitter.com                                         236390      1
amazon.com\__CLIENT_ERROR__                          99984      2
www.facebook.com                                     56837      3
twitter.com\__CLIENT_ERROR__                         45800      4
www.youtube.com                                      43479      5
www.instagram.com                                    26276      6
t.co\__CONNECTIONPOOL_ERROR__                        24968      7
t.co\__CLIENT_ERROR__                                24695      8
instagram.com\__CLIENT_ERROR__                       19391      9
vine.co                                              15101     10
www.redbubble.com                                    11942     11
www.cafepress.com                                    11571     12
youtube.co

In [23]:
print(df2.head())

                   id                                               text  \
1  593814612820119552  sanfranciscoba enoughsenough surprise woody al...   
2  593659318479687680            sippey one paragraph review new macbook   
3  593579247110782976  mmflint dont ask rts gen friend frakintosh col...   
4  593578595370438657  acts messages people say still matter pls tell...   
5  593578251722690560  live untreated ptsd feel like might die moment...   

   user_id  num_hashtags  num_mentions  num_urls          finalurl  \
1   678033           1.0           1.0       1.0              t.co   
2   678033           0.0           1.0       1.0        medium.com   
3   678033           0.0           2.0       1.0  www.gofundme.com   
4   678033           1.0           1.0       1.0  www.gofundme.com   
5   678033           1.0           0.0       1.0  www.gofundme.com   

                      url  followers_count  friends_count  \
1  http://t.co/98fWa0t873           7065.0         1822.0   


In [49]:
mask=df2['size']<500
df2.loc[mask,'size'] =np.nan
df2.dropna(inplace=True)
print(df2['size'].unique())

[  8822. 236390.    544.   1708.   4964.  24695.  26276.  56837.    606.
  43479.   1715.    534.  10137.  45800.    619.    836.   2315.    585.
   2316.    535.   2536.   1543.   1679.   1871.    536.  19391.    677.
  15101.   1022.    566.   2576.    946.    702.    587.   1088.   1369.
    614.    567.   1169.  99984.    870.    515.   2740.   2459.    651.
   3574.    811.   1138.  24968.   6064.   5634.    901.   1197.   1458.
    758.   2401.  11942.    556.   1324.    840.   1607.   1262.   3570.
    875.   2604.    894.   1802.    583.   4742.  11571.    823.    513.
   2381.    849.   1333.   2831.    644.    514. 454996.    563.   1019.
   1271.   2213.    503.    560.    760.   1360.    591.    769.   3223.
   5285.   7214.    597.   3798.    500.   1216.    520.    971.   4225.
   2052.    540.    641.    956.   2993.   2330.   4488.    700.    573.
    688.    668.    890.    615.    640.    504.    642.    767.    753.]


In [50]:
print(len(df2))
print(df2['index'].value_counts())

1251018
0      454996
1      236390
2       99984
3       56837
4       45800
5       43479
6       26276
7       24968
8       24695
9       19391
10      15101
11      11942
12      11571
13      10137
14       8822
15       7214
16       6064
17       5634
18       5285
19       4964
20       4742
21       4488
22       4225
23       3798
24       3574
25       3570
26       3223
27       2993
28       2831
29       2740
        ...  
91        640
92        619
93        615
94        614
95        606
96        606
97        597
98        591
99        587
100       585
101       583
102       573
103       567
104       566
105       563
106       560
107       556
108       544
109       540
110       536
111       536
112       535
113       534
114       520
115       515
116       514
117       513
118       504
119       503
120       500
Name: index, Length: 121, dtype: int64


In [51]:
finaldf=pd.DataFrame()
print(finaldf)

Empty DataFrame
Columns: []
Index: []


In [52]:
scope=df2['index'].max()
print(scope)
for i in range(scope+1):
    print(i)

w1=0.8
w2=0.5
w3 = 1

120
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120


In [53]:
print(df2.columns)

Index(['id', 'text', 'user_id', 'num_hashtags', 'num_mentions', 'num_urls',
       'finalurl', 'url', 'followers_count', 'friends_count', 'created_at',
       'verified', 'protected', 'reputation', 'social', 'isSpam', 'size',
       'index'],
      dtype='object')


In [139]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df2['text'])
transformer = TfidfTransformer()
tfidf = transformer.fit_transform(X)
df2['tfidf']=list(tfidf)

In [54]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score
for i in range(scope+1):
    test=df2[df2['index']==i]
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(test['text'])
    transformer = TfidfTransformer()
    tfidf = transformer.fit_transform(X)
   
    def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
        distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
        return np.mean(distances)

    t_data=TruncatedSVD(n_components=2).fit_transform(tfidf)
    test['tfidf']=tfidf
    print(t_data.shape)
    k_means=KMeans()
    clusters=k_means.fit_predict(tfidf)
    centroids = k_means.cluster_centers_
    centroids=TruncatedSVD(n_components=2).fit_transform(centroids)
    c_mean_distances = []
    for i, (cx, cy) in enumerate(centroids):
        mean_distance = k_mean_distance(t_data, cx, cy, i, clusters)
        c_mean_distances.append(mean_distance)
    test1=pd.DataFrame(clusters)

    test2 = pd.DataFrame({'size':test1.groupby(0).size()})
    test2 = test2.sort_index( ascending=True)
    test2['cluster']=c_mean_distances[0:len(test2)]
    test2['selfsimilar']=test2['size']**w1*test2['cluster']**w2/len(test2)*w3
    test['self_similarity']=test2['selfsimilar'].sum()
    test['num_hashtags']=test['num_hashtags'].sum()/test['size']
    test['num_mentions']=test['num_mentions'].sum()/test['size']
    finaldf=finaldf.append(test)








D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(454996, 2)


D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(236390, 2)


...

(503, 2)


D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


(500, 2)


D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [34]:
print(finaldf.head())



                        id                                               text  \
134606  319447633200574464  poignant look different world squidoo lens hom...   
134807  302865619638972418  pretty cool squidoo lens black rhinoceros verg...   
201555  301345366094270464           goodbye wheat belly hello healthier life   
302417  592051457345658880  tips creating schedule working home rebeccagra...   
302715  578988886749130753  use humor therapy stress stresstherapy rebecca...   

          user_id  num_hashtags  num_mentions  num_urls  \
134606   14391111      0.011703      0.014081       1.0   
134807   14391111      0.011703      0.014081       1.0   
201555  515162435      0.011703      0.014081       1.0   
302417   16957615      0.011703      0.014081       1.0   
302715   16957615      0.011703      0.014081       1.0   

                             finalurl                     url  \
134606  hubpages.com\__CLIENT_ERROR__  http://t.co/iqpVjy13PJ   
134807  hubpages.com\__CLIEN

In [55]:
spamList=open("D:\study\python\Spamdetection\\blacklist.txt",encoding='utf-8').read()
spamList=spamList.split()
def spamWordRatio(word):
    word=word.split()
    count=0
    for i in word:
        if i in spamList:
            count=count+1
    return count/len(word)
finaldf['spamRatio']=finaldf['text'].apply(spamWordRatio)
print(finaldf['spamRatio'].value_counts())

0.500000    162147
0.000000    143724
0.250000     88592
0.333333     88379
0.666667     83697
1.000000     68816
0.600000     68694
0.750000     68379
0.400000     60893
0.200000     35857
0.800000     30012
0.428571     23269
0.285714     20797
0.571429     20652
0.166667     19114
0.833333     18813
0.545455     16146
0.625000     15575
0.375000     14226
0.142857     13667
0.714286     12232
0.444444     11217
0.857143     10791
0.125000     10273
0.222222      9301
0.555556      9019
0.300000      8834
0.727273      7960
0.272727      7208
0.111111      6962
             ...  
0.352941        35
0.923077        25
0.928571        24
0.529412        17
0.687500        15
0.105263        15
0.157895        14
0.470588        11
0.052632        11
0.210526        11
0.388889         9
0.588235         6
0.812500         6
0.150000         6
0.368421         6
0.705882         6
0.647059         5
0.315789         5
0.050000         5
0.722222         4
0.350000         4
0.047619    

In [57]:
# print(finaldf.dropna(axis=0,inplace=True))
print(finaldf.columns)

Index(['id', 'text', 'user_id', 'num_hashtags', 'num_mentions', 'num_urls',
       'finalurl', 'url', 'followers_count', 'friends_count', 'created_at',
       'verified', 'protected', 'reputation', 'social', 'isSpam', 'size',
       'index', 'tfidf', 'self_similarity', 'spamRatio'],
      dtype='object')


In [56]:
finaldf['created_at']=pd.to_datetime(finaldf['created_at'],utc=True).dt.strftime('%Y')
finaldf['created_at']=finaldf['created_at'].astype(int)


In [59]:
finaldf.reset_index(inplace=True)
finaldf.drop(index=[92120, 892417, 892475],inplace=True)

In [60]:
np.where(np.isinf(finaldf['social']))


(array([], dtype=int64),)

In [168]:
def addheader(url):
    return "http://"+url
x=finaldf.groupby('finalurl')['isSpam'].mean()
print(x)

finalurl
                                                           0.010288
21stsoft.com\__CLIENT_ERROR__                              1.000000
aadecorativeconcrete.com                                   1.000000
affects-fp7.eu\__CLIENT_ERROR__                            0.000000
amazon.com\__CLIENT_ERROR__                                0.993869
amazon.de\__CLIENT_ERROR__                                 1.000000
amp.twimg.com                                              0.013629
archive.piovegovernoladro.info\__CONNECTIONPOOL_ERROR__    1.000000
ask.fm\__CLIENT_ERROR__                                    0.021299
atomix.vg                                                  0.000000
beppegrillo.it\__CLIENT_ERROR__                            1.000000
biggboymgmt.sharedby.co\__CONNECTIONPOOL_ERROR__           1.000000
bizj.us                                                    0.006861
bleacherreport.com                                         0.099846
blog.stratcommunications.com           

In [62]:
finaldf.drop(['level_0'],axis=1,inplace=True)
print(finaldf.head())

                   id                                               text  \
0  319447633200574464  poignant look different world squidoo lens hom...   
1  302865619638972418  pretty cool squidoo lens black rhinoceros verg...   
2  301345366094270464           goodbye wheat belly hello healthier life   
3  592051457345658880  tips creating schedule working home rebeccagra...   
4  578988886749130753  use humor therapy stress stresstherapy rebecca...   

     user_id  num_hashtags  num_mentions  num_urls  \
0   14391111      0.011703      0.014081       1.0   
1   14391111      0.011703      0.014081       1.0   
2  515162435      0.011703      0.014081       1.0   
3   16957615      0.011703      0.014081       1.0   
4   16957615      0.011703      0.014081       1.0   

                        finalurl                     url  followers_count  \
0  hubpages.com\__CLIENT_ERROR__  http://t.co/iqpVjy13PJ          22452.0   
1  hubpages.com\__CLIENT_ERROR__  http://t.co/iqpVjy13PJ        

In [77]:
finaldf['finalurl']=finaldf['finalurl'].astype(str)
print(finaldf.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1251015 entries, 0 to 1251017
Data columns (total 21 columns):
id                 1251015 non-null int64
text               1251015 non-null object
user_id            1251015 non-null int64
num_hashtags       1251015 non-null float64
num_mentions       1251015 non-null float64
num_urls           1251015 non-null float64
finalurl           1251015 non-null object
url                1251015 non-null object
followers_count    1251015 non-null float64
friends_count      1251015 non-null float64
created_at         1251015 non-null int32
verified           1251015 non-null float64
protected          1251015 non-null float64
reputation         1251015 non-null float64
social             1251015 non-null float64
isSpam             1251015 non-null int64
size               1251015 non-null float64
index              1251015 non-null int64
tfidf              1251015 non-null object
self_similarity    1251015 non-null float64
spamRatio          12

In [194]:
from spam_lists import SPAMHAUS_DBL

from spam.surbl import SurblChecker
urls_to_test = (
'http://medium.com'
 


)
A=SPAMHAUS_DBL.lookup('dbltest.com')

print(A)

Timeout: The DNS operation timed out after 30.00592541694641 seconds

In [64]:
# finaldf.drop(['url','text','level_0'],axis=1,inplace=True)
# finaldf.drop(['finalurl'],axis=1,inplace=True)
final3=finaldf.drop(['tfidf','url'],axis=1)
final3.dropna(axis=0,inplace=True)
final3.to_csv('D:/study/Fake/finally.csv')


In [195]:
print(final3.columns)
new=final3.drop(['text'],axis=1)
new=pd.get_dummies(new)

Index(['id', 'text', 'user_id', 'num_hashtags', 'num_mentions', 'num_urls',
       'finalurl', 'followers_count', 'friends_count', 'created_at',
       'verified', 'protected', 'reputation', 'social', 'isSpam', 'size',
       'index', 'self_similarity', 'spamRatio'],
      dtype='object')


In [189]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
# new=new.drop(['tfidf'],axis=1)

y = new['isSpam']
X = new.drop(['isSpam'], axis=1)
X_mat=X.as_matrix()
rs=20

X_train, X_test, y_train, y_test = train_test_split(X_mat, y
                                            , stratify=y,test_size=0.2,random_state=rs
                                                    )
model = RandomForestClassifier(max_depth=7)
model.fit(X_train,y_train)
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))
y_pred = model.predict(X_test)

report =  classification_report(y_test, y_pred) 


print(report)


D:\study\python\venv\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':


D:\study\python\venv\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Train accuracy: 0.9531670283729612


Test accuracy: 0.9524066458036075


              precision    recall  f1-score   support

           0       0.93      0.94      0.94     95021
           1       0.96      0.96      0.96    155182

   micro avg       0.95      0.95      0.95    250203
   macro avg       0.95      0.95      0.95    250203
weighted avg       0.95      0.95      0.95    250203



In [185]:
model = KNeighborsClassifier()
model.fit(X_train,y_train)
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))
y_pred = model.predict(X_test)

report =  classification_report(y_test, y_pred) 


print(report)

Train accuracy: 0.9282682461840985


Test accuracy: 0.9040299276987086


              precision    recall  f1-score   support

           0       0.90      0.84      0.87     95021
           1       0.91      0.94      0.92    155182

   micro avg       0.90      0.90      0.90    250203
   macro avg       0.90      0.89      0.90    250203
weighted avg       0.90      0.90      0.90    250203



In [186]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train,y_train)
print("Train accuracy:", model.score(X_train, y_train))
print("Test accuracy:", model.score(X_test, y_test))
y_pred = model.predict(X_test)

report =  classification_report(y_test, y_pred) 


print(report)

Train accuracy: 0.7147506224945345


Test accuracy: 0.7153631251423844


              precision    recall  f1-score   support

           0       0.83      0.31      0.45     95021
           1       0.70      0.96      0.81    155182

   micro avg       0.72      0.72      0.72    250203
   macro avg       0.76      0.64      0.63    250203
weighted avg       0.75      0.72      0.67    250203

